In [45]:
import pandas as pd
import os as os
import matplotlib.pyplot as plt

In [46]:
df = pd.read_csv('~/Downloads/OnlineRetail.csv', sep=';')


In [47]:
# Convert data types
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'].str.replace(',', '.'), errors='coerce')
df['CustomerID'] = pd.to_numeric(df['CustomerID'], errors='coerce')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%d.%m.%Y %H:%M')


df.dtypes
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


(541909, 8)

In [49]:
# Clean the data
df = df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum())

df = df.dropna(subset=['CustomerID', 'UnitPrice'])
df = df[df['Quantity'] >= 0]

# Calculate Monetary value
df['Monetary'] = df['Quantity'] * df['UnitPrice']


# Calculate the latest date for recency calculation
latest_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

df.dtypes

df.shape
print(len(df))


0
392732


In [53]:
# Recency Calculation
recency = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
recency['Recency'] = (latest_date - recency['InvoiceDate']).dt.days
print(len(recency))


# Frequency Calculation
frequency = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
frequency.columns = ['CustomerID', 'Frequency']
print(len(frequency))

# Merge Recency and Frequency
rfm = pd.merge(recency[['CustomerID', 'Recency']], frequency, on='CustomerID')
print(len(rfm))

# Monetary Calculation
monetary = df.groupby('CustomerID')['Monetary'].sum().reset_index()


# Merge Monetary
rfm = pd.merge(rfm, monetary, on='CustomerID')
print(rfm.shape)

4339
4339
4339
(4339, 4)


In [51]:
# Compute Quantiles
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75], numeric_only=True)


# Define RFM segmentation functions
def RScore(x, p, d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

def FMScore(x, p, d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1


# Apply segmentation
rfm['R'] = rfm['Recency'].apply(RScore, args=('Recency', quantiles,))
rfm['F'] = rfm['Frequency'].apply(FMScore, args=('Frequency', quantiles,))
rfm['M'] = rfm['Monetary'].apply(FMScore, args=('Monetary', quantiles,))

# Concatenate RFM segment codes
rfm['RFM_Segment'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)

# Display the final RFM segmentation
rfm


,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Segment
0,12346.0,326,1,77183.60,4,4,1,441
1,12347.0,2,7,4310.00,1,1,1,111
2,12348.0,75,4,1797.24,3,2,1,321
3,12349.0,19,1,1757.55,2,4,1,241
4,12350.0,310,1,334.40,4,4,3,443
...,...,...,...,...,...,...,...,...
4334,18280.0,278,1,180.60,4,4,4,444
4335,18281.0,181,1,80.82,4,4,4,444
4336,18282.0,8,2,178.05,1,3,4,134
4337,18283.0,4,16,2045.53,1,1,1,111
